### Лабораторная работа №7
Выполнила Карасева Екатерина M33351

#### Обработка последовательностей

Обработка текста

In [1]:
import re


def clearing_file(input_filename, output_filename):
    with open(input_filename, 'r', encoding='utf-8') as input_file:
        with open(output_filename, 'w', encoding='utf-8') as output_file:
            text = input_file.read()
            text = text.lower()
            text = text.replace('ё', 'е')
            text = text.replace('!', '.')
            text = text.replace('?', '.')
            text = ''.join(ch if bool(re.search('[а-я. \n]', ch)) else '' for ch in text)
            text = re.sub(r'\s+', r' ', text)
            text = re.sub(r' *\n *', r'', text)
            text = re.sub(r'\n\.\n', r'', text)
            text = re.sub(r'\.+', r'.', text)

            output_file.write(text)
            return text


file = 'text2.txt'
clear_file = 'clear_text2.txt'

clear_file_text = clearing_file(file, clear_file)

In [2]:
to_one_hot = {}
from_one_hot = {}
a = ord('а')
len_d = 34
ind = 0
for litter in [chr(i) for i in range(a, a + len_d - 2)] + [' ', '.']:
    l = [0] * len_d
    l[ind] = 1
    to_one_hot[litter] = l
    from_one_hot[str(l)] = litter
    ind += 1


def get_one_hot(text):
    return list(map(lambda ch: to_one_hot[ch], text))

In [3]:
from nltk import ngrams
import numpy as np

len_ngrams = 10


def get_ngrams(arr, n):
    ngrams_text = list(ngrams(arr, n + 1))
    size_ngrams = len(ngrams_text)
    x = np.empty(shape=[size_ngrams, n, len_d])
    y = np.empty(shape=[size_ngrams, len_d])
    for window, i in zip(ngrams_text, range(size_ngrams)):
        x[i] = np.array(window[:n])
        y[i] = np.array(window[-1])
    return x, y


In [4]:
from tensorflow import keras
from keras import layers

obj, target = get_ngrams(get_one_hot(clear_file_text), len_ngrams)

model = keras.Sequential()
model.add(keras.Input(shape=obj.shape[1:]))
model.add(layers.LSTM(256, return_sequences=True))
model.add(layers.LSTM(256))
model.add(layers.Dense(target.shape[1], activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(obj, target, batch_size=128, epochs=30)

Epoch 1/30
199/199 [==============================] - 28s 126ms/step - loss: 3.0401 - accuracy: 0.1849

In [5]:
def random_string_generating(filename, predict):
    with open(filename, 'r', encoding='utf-8') as f:
        text = f.read()
        sentences = text.split('.')
        sentences = list(filter(lambda x: len(x) >= len_ngrams, sentences))
        sentences_count = len(sentences)

        index = np.random.randint(0, sentences_count - 1)

        text = sentences[index][:len_ngrams]
        result_text = text
        cur_text = text

        for i in range(100):
            prediction = predict(cur_text)
            result_text += prediction
            cur_text = cur_text[1:] + prediction

        return sentences[index], text, result_text

def lstm_predict(text):
    one_hot = get_one_hot(text)
    test = [one_hot]
    prediction = np.argmax(model.predict(test))
    one_hot_pred = [0] * len_d
    one_hot_pred[prediction] = 1
    return from_one_hot[str(one_hot_pred)]

In [14]:
full_sentence, first_part, predicted_sentence = random_string_generating(clear_file, lstm_predict)

print(full_sentence)
print(first_part)
print(predicted_sentence)

1/1 [==============================] - 0s 31ms/step
 больно когда человек уходит и ты пытаешься его забыть но это не выходит
 больно ко
 больно когда судьба тебе говорит все. больно когда судьба тебе говорит все. больно когда судьба тебе говорит 


In [7]:
full_sentence, first_part, predicted_sentence = random_string_generating(clear_file, lstm_predict)

print(full_sentence)
print(first_part)
print(predicted_sentence)

1/1 [==============================] - 0s 64ms/step
 привет гуки обнимает за шею и вздыхает крепкий одекалон парня как дела
 привет гу
 привет гуки обнимает за шею и чмокает в губы. привет милый шепчет он. пойдем поедим моего любимого человека о


In [15]:
full_sentence, first_part, predicted_sentence = random_string_generating(clear_file, lstm_predict)

print(full_sentence)
print(first_part)
print(predicted_sentence)

1/1 [==============================] - 0s 31ms/step
 чонгук кивает слегка головой и замыкает глядя на молчаливого тэхена и снова я думаю не так да
 чонгук ки
 чонгук кивает слегка головой и выставляет дрожащую губы и выпускает поток слез. прости меня боже чонгук выдых


### Марковские цепи

In [9]:
from collections import defaultdict

def data_to_markov(data):
    ngrams_text = list(ngrams(data, len_ngrams + 1))
    size_ngrams = len(ngrams_text)

    x = []
    y = []
    for window, index in zip(ngrams_text, range(size_ngrams)):
        x.append(''.join(window[:len_ngrams]))
        y.append(window[-1])

    return np.array(x), np.array(y)

def get_markov(x_train, y_train):
    nodes_markov = defaultdict(lambda: defaultdict(lambda: 0))
    for sentence, symbol in zip(x_train, y_train):
        nodes_markov[sentence][symbol] += 1
    return nodes_markov

x_markov, y_markov = data_to_markov(clear_file_text)
nodes = get_markov(x_markov, y_markov)

In [16]:
def markov_predict(text):
    node = nodes[text]
    weights = np.array([w for w in node.values()])
    if len(weights) == 0:
        return '.'
    index = node[np.argmax(weights)]
    return [w for w in node][index]

full_sentence, first_part, predicted_sentence = random_string_generating(clear_file, markov_predict)

print(full_sentence)
print(first_part)
print(predicted_sentence)

 все будет хорошо я уткнулся ему в плечо и обнял в ответ
 все будет
 все будет повторятся как и сегодня. не сказать что я один. у меня есть верный и любимый друг и даже единствен


In [17]:
full_sentence, first_part, predicted_sentence = random_string_generating(clear_file, markov_predict)

print(full_sentence)
print(first_part)
print(predicted_sentence)

 дело было совсем не в ревности а в отношении друг друга
 дело было
 дело было совсем не в ревности а в отношении друг другу в чувствах. это был не забываемый день который я не з


In [19]:
full_sentence, first_part, predicted_sentence = random_string_generating(clear_file, markov_predict)

print(full_sentence)
print(first_part)
print(predicted_sentence)

 ким промаргивается и подходит ближе упираясь руками сжимая сильно поручень моста
 ким прома
 ким промаргивается и уходит прочь с коридора даже не оглядываясь на друга что смотрит ему в след и грустно ул
